In [ ]:
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.svm import SVC
from sklearn.metrics import classification_report
from skimage.feature import local_binary_pattern
from src.features.lbp_extractor import LBPExtractor



In [ ]:
REAL_LABEL = 0
IMPOSTER_LABEL = 1
IMG_SIZE = (128, 128)

In [ ]:
attack_dir = r"/content/drive/MyDrive/Detectedface/ImposterFace"
real_dir = r"/content/drive/MyDrive/Detectedface/ClientFace"
extractor = LBPExtractor(grid_x=4, grid_y=4)
x = []
y = []

In [ ]:
def load_data(directory, label):
    count = 0
    # Walk through directory to find all image files
    for root, _, files in os.walk(directory):
        for filename in files:
            if filename.lower().endswith(('.jpg', '.png', '.jpeg', '.bmp')):
                img_path = os.path.join(root, filename)
                img = cv2.imread(img_path)
                if img is not None:
                    try:
                        # Resize and Extract
                        img_resized = cv2.resize(img, IMG_SIZE)
                        feat = extractor.extract(img_resized)
                        x.append(feat)
                        y.append(label)
                        count += 1
                    except Exception as e:
                        print(f"Error processing {filename}: {e}")
    return count

In [ ]:
print("Starting data loading...")
n_real = load_data(real_dir, REAL_LABEL)
print(f"Loaded {n_real} real images.")
n_imposter = load_data(attack_dir, IMPOSTER_LABEL)
print(f"Loaded {n_imposter} imposter images.")

In [ ]:
X_data = np.array(x)
Y_data = np.array(y)
# Split
xtr, xte, ytr, yte = train_test_split(X_data, Y_data, test_size=0.2, random_state=21)
# A. Feature Scaling
scaler = StandardScaler()
xtr = scaler.fit_transform(xtr)
xte = scaler.transform(xte)


In [ ]:
pca = PCA(n_components=0.95, random_state=21)
xtr = pca.fit_transform(xtr)
xte = pca.transform(xte)
print(f"Feature dims reduced from {scaler.n_features_in_} to {pca.n_components_} components")


In [ ]:
param_grid = {
    'C': [0.01, 0.1, 1, 10],   
    'gamma': ['scale', 'auto'],
    'kernel': ['rbf']
}
grid = GridSearchCV(SVC(class_weight="balanced", probability=True),
                    param_grid, cv=3, n_jobs=-1, verbose=1)
grid.fit(xtr, ytr)
best_model = grid.best_estimator_
print(f"Best Parameters: {grid.best_params_}")

In [ ]:
y_preds = best_model.predict(xte)
print("\n" + classification_report(yte, y_preds, target_names=['Real', 'Spoof']))
